Yue write this code at 2023/08/10

The aim is to realize the virtual compliance control of the leg motion.

# mathmatic

some calculation test to verify our mathmatic formula

In [ ]:
import numpy as np
import math
from math import sin,cos,pi
import matplotlib.pyplot as plt
L=80

In [ ]:
def Jocobi(theta0,theta1):
    J=np.array([[-L*sin(theta0),L*sin(theta1)],[L*cos(theta0),L*cos(theta1)]])
    return J
J=Jocobi(pi/4,pi/4)
print(J)

In [ ]:
a=np.array([[1,2],[3,4]])
b=np.array([1,2])
c=np.dot(a.T,b)
print(c)

In [ ]:
F=np.array([1,0])
print(F.shape)
tau=np.dot(J.T,F) 
print(tau)

In [ ]:
ORIGIN_POS=[0.177-0.25,0.041+0.25]

def motor_pos_at(x,y):
    ad=np.sqrt(x**2+y**2)

    if ad>=2*LEG or ad<=0 or y<0:
        print('Error:position out of reach')
        return ORIGIN_POS
    else:
        beta=math.acos(ad/(2*LEG))%math.pi
        
        if x==0:
            alpha=math.pi/2
        else:
            alpha=math.atan(y/x)%math.pi

        theta0=alpha-beta
        theta1=alpha+beta

        pos_0=ORIGIN_POS[0]+theta0/(2*math.pi)
        pos_1=ORIGIN_POS[1]-theta1/(2*math.pi)

        return pos_0,pos_1
    
def get_end_pos(encoder0,encoder1):
    theta0=(encoder0-ORIGIN_POS[0])*2*pi
    theta1=(encoder1-ORIGIN_POS[1]-0.5)*2*pi
    alpha=(theta0+theta1)/2
    beta=(theta1-theta0)/2
    x=2*L*sin(alpha)*sin(beta)
    y=2*L*sin(alpha)*cos(beta)
    return x,y

pos0,pos1=motor_pos_at(10,50)
print(get_end_pos(pos0,pos1))

# control

In [1]:
import odrive
from odrive.enums import *
import time
import math
from math import sin,cos,pi
import matplotlib.pyplot as plt
from scipy.signal import butter,filtfilt
from IPython.display import clear_output
import numpy as np
import serial
import pickle
from controller import COMPLIANCE_CONTROLLER,POSITION_CONTROLLER
def check(my_drive):
    if my_drive.axis1.error == 0:
        print('Everything is fine')
    else:
        print('Error!')


In [2]:
# Find a connected ODrive (this will block until you connect one)
print("finding an odrive...")
my_drive = odrive.find_any()
print("Found! and the bus voltage is " + str(my_drive.vbus_voltage) + "V")
print(my_drive.axis0.motor.current_control.i_gain,my_drive.axis1.motor.current_control.i_gain)
my_drive.axis1.motor.current_control.i_gain = 4
my_drive.axis0.motor.current_control.i_gain = 4
print(my_drive.axis0.motor.current_control.i_gain,my_drive.axis1.motor.current_control.i_gain)


my_drive.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
my_drive.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL


check(my_drive)
print('*'*50)


finding an odrive...
Found! and the bus voltage is 28.166015625V
80.16718292236328 82.6633529663086
4.0 4.0
Everything is fine
**************************************************


In [5]:
print(my_drive.motor.config)

enable_uart = True (bool)
uart_baudrate = 115200 (int)
enable_i2c_instead_of_can = False (bool)
enable_ascii_protocol_on_usb = True (bool)
max_regen_current = 0.05000000074505806 (float)
brake_resistance = 0.550000011920929 (float)
dc_bus_undervoltage_trip_level = 8.0 (float)
dc_bus_overvoltage_trip_level = 59.92000198364258 (float)
enable_dc_bus_overvoltage_ramp = False (bool)
dc_bus_overvoltage_ramp_start = 59.92000198364258 (float)
dc_bus_overvoltage_ramp_end = 59.92000198364258 (float)
dc_max_positive_current = inf (float)
dc_max_negative_current = -0.10000000149011612 (float)
gpio1_pwm_mapping:
  endpoint = (0, 0) (RemoteProperty)
  min = 0.0 (float)
  max = 0.0 (float)
gpio2_pwm_mapping:
  endpoint = (0, 0) (RemoteProperty)
  min = 0.0 (float)
  max = 0.0 (float)
gpio3_pwm_mapping:
  endpoint = (0, 0) (RemoteProperty)
  min = 0.0 (float)
  max = 0.0 (float)
gpio4_pwm_mapping:
  endpoint = (0, 0) (RemoteProperty)
  min = 0.0 (float)
  max = 0.0 (float)
gpio3_analog_mapping:
  endp

In [10]:
print(my_drive.axis0.motor.config.torque_constant,my_drive.axis1.motor.config.torque_constant)
my_drive.axis0.controller.config.control_mode=1
my_drive.axis1.controller.config.control_mode=1

1.0 1.0


In [ ]:
# get the load_Cell initail value
ser = serial.Serial(  # 下面这些参数根据情况修改
  port='COM4',# 串口
  baudrate=57600,# 波特率
  parity=serial.PARITY_ODD,
  stopbits=serial.STOPBITS_TWO,
  bytesize=serial.SEVENBITS
)

# my_drive.axis1.controller.config.control_mode = 3
# my_drive.axis1.controller.input_pos=94/360
time.sleep(1)

#read the initial value of now
t0=time.monotonic()
delta=0
while delta<5:
  delta=time.monotonic()-t0
  data = ser.readline()
#   clear_output()
  try:
    print(int(data))
    cell_ini=int(data)
  except:
    print(data)

print('initial value of load cell',cell_ini)

## static test

just use the load cell to test the force output and compare to the force it supposed to be as a compliance part. Only on one static position


In [ ]:
K=2.09e-5#unit: N
all_time=[]
all_virtual_force=[]
all_input=[]
all_measured_force=[]
all_pos=[]
max_current=100


sample_num=0

t0=time.monotonic()
delta=time.monotonic()-t0


while delta<15:
    delta=time.monotonic()-t0
    pos_mea=[my_drive.axis0.encoder.pos_circular,my_drive.axis1.encoder.pos_circular]
    J=Jocobi(pos_mea[0],pos_mea[1])
    
    #controller
    virtual_force=[-abs(4*sin(delta-0.5)),0]#unit N
    motor_torque=np.dot(J.T,virtual_force)
    motor_input=[]
    for i in range(2):
        if motor_torque[i]>0:
            motor_input.append(min(motor_torque[i]/TORQUE_CONSTANT,max_current))
        else:
            motor_input.append(max(motor_torque[i]/TORQUE_CONSTANT,-max_current))
    my_drive.axis0.controller.input_torque = motor_input[0]
    my_drive.axis1.controller.input_torque = motor_input[1]

    #read the load cell
    load_cell = ser.readline() #will stuck here until receive data--so its frequency determine the whole test system frequency
    try:
        int(load_cell)
        measured_force=(int(load_cell)-cell_ini)*K
    except:
        measured_force=0

    #record data
    all_time.append(delta)
    all_input.append(motor_input)
    all_virtual_force.append(virtual_force)
    all_measured_force.append(measured_force)
    all_pos.append(pos_mea)
    
    #display the data
    sample_num+=1
    if sample_num%5==0:
        clear_output()
        print('VIRTUAL_FORCE: {} \nMOTOR INPUT: {} \nMEASURED_FORCE: {}'.format(virtual_force,motor_input,measured_force))
        check(my_drive)

my_drive.axis0.controller.input_torque = 0
my_drive.axis1.controller.input_torque = 0
check(my_drive)

test_data=[all_time,all_input,all_virtual_force,all_measured_force]
filepath='./test/virtual_force_static_with_end_at_{:.1f}_{:.1f}.pkl'.format(get_end_pos(all_pos[-1][0],all_pos[-1][1])[0],get_end_pos(all_pos[-1][0],all_pos[-1][1])[1])
pickle.dump(test_data,open(filepath,'wb'))


In [ ]:
all_virtual_force_x=-np.array(all_virtual_force)[:,0]
all_virtual_force_y=-np.array(all_virtual_force)[:,1]

plt.plot(all_time,all_virtual_force_x,linewidth=3,label='virtual force_x')
plt.plot(all_time,all_virtual_force_y,linewidth=3,label='virtual force_y')

plt.plot(all_time,all_measured_force,linewidth=3,linestyle='dashed',label='measured_force')
plt.xlabel('time(s)')
plt.ylabel('force(N)')
plt.title('end position: x={:.2f} y={:.2f}'.format(get_end_pos(all_pos[-1][0],all_pos[-1][1])[0],get_end_pos(all_pos[-1][0],all_pos[-1][1])[1]))
plt.legend()
plt.ylim(0,6)
plt.show()



## dynamic test

almost the same with static test, but I mannually change the position to get the relationship between output force and position to verify the compliance

In [11]:
# control the end stay at the origin point
# letsgo(motor_pos_at(0,100),2)
print('ready to do the compliance control')
time.sleep(2)

my_drive.axis0.controller.config.control_mode=1
my_drive.axis1.controller.config.control_mode=1

controller=COMPLIANCE_CONTROLLER([0.1,0.1],[3e-3,3e-3])

K=2.09e-5#unit: N
ref_pos=[10,100]

all_time=[]
all_pos=[]
all_input=[]
all_speed=[]
sample_num=0

t0=time.monotonic()
time.sleep(0.1)
delta=time.monotonic()-t0

while delta<30:
    #controller
    delta,measured_pos,speed,motor_input=controller.control(my_drive,ref_pos,all_pos,all_time,t0)

    #record data
    all_time.append(delta)
    all_pos.append(measured_pos)
    all_input.append(motor_input)
    all_speed.append(speed)

    #display the data
    sample_num+=1
    if sample_num%20==0:
        clear_output()
        print('STIFFNESS:{} \nREF_POSITION:{} \nMEASURED POSITION: {} \nMOTOR INPUT: {} \nCONTROL FREQUENCTY: {}'
              .format(controller.stiffness,np.around(ref_pos,decimals=1),np.around(measured_pos,decimals=1)
                      ,np.around(motor_input,decimals=1),sample_num/delta))
        print('SPEED: {}'.format(np.around(speed,decimals=1)))
        check(my_drive)


test_data=[all_time,all_pos,all_input]
filepath='./test/leg_impedance_control_at_{}with_k={}.pkl'.format(ref_pos,controller.stiffness)
pickle.dump(test_data,open(filepath,'wb'))

STIFFNESS:[0.1, 0.1] 
REF_POSITION:[ 10 100] 
MEASURED POSITION: [ 11.6 114. ] 
MOTOR INPUT: [-21.4 -21.9] 
CONTROL FREQUENCTY: 226.23377490073037
SPEED: [-0.6 -2.8]
Everything is fine


In [ ]:
my_drive.axis0.controller.input_torque = 0
my_drive.axis1.controller.input_torque = 0

In [ ]:
all_speed_array_x=np.array(all_speed)[:,0]
all_speed_array_y=np.array(all_speed)[:,1]

plt.plot(all_time,all_speed_array_x,label='x')
plt.plot(all_time,all_speed_array_y,label='y')
plt.legend()
plt.xlabel('time(s)')
plt.ylabel('end speed(mm/s)')
# plt.xlim(80,90)
plt.show()

In [ ]:
all_pos_array_x=np.array(all_pos)[:,0]
all_pos_array_y=np.array(all_pos)[:,1]

plt.plot(all_time,all_pos_array_x,label='x')
plt.plot(all_time,all_pos_array_y,label='y')
plt.legend()
plt.xlabel('time(s)')
plt.ylabel('end position(mm)')
# plt.xlim(10,20)
plt.show()

### plot measured force and virtual force``

In [ ]:

print('control fre={}'.format(len(all_time)/all_time[-1]))
print(all_pos[0],all_input[0])
all_elongation=np.array(all_pos)[:,0]-desired_end_pos[0]
plt.scatter(all_elongation,all_measured_force,s=15,label='measured force')

all_supposed_force=all_elongation*end_stiffness[0]
plt.plot(all_elongation,all_supposed_force,'r',label='virtual spring force')

plt.title('stiffness={} desired_pos={}'.format(end_stiffness,desired_end_pos))
plt.ylabel('measured force(N)')
plt.xlabel('end position error x(mm)')
plt.legend()
plt.ylim(-1,5)
plt.show()


In [ ]:
all_input_array=np.array(all_input)
all_supposed_input_array=np.array(all_supposed_torque)/3.6
plt.plot(all_time,all_input_array[:,0],label='AXIS0 input')
plt.plot(all_time,all_supposed_input_array[:,0],label='AXIS0 supposed input')
plt.title('stiffness={} desired_pos={}'.format(end_stiffness,desired_end_pos))
plt.ylabel('motor input current(A/100)')
plt.xlabel('time(s)')
# plt.xlim(18,22)
plt.legend()
plt.show()
